plus slice timing + 16 mars normalisation des images structurelles segmentees en tissus
premier notebook avec nouvelle version de marc spm

## preliminaries

In [1]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.rapidart as ra      # artifact detection
import nipype.algorithms.modelgen as model   # model specification
import os                                    # system functions


In [2]:
spm_standalone='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/spm12/run_spm12.sh'
mcr='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/mcr2016/v91'

In [3]:
from nipype.interfaces import spm

# essai du mercredi soir matlab_cmd = ' '.join([spm_standalone,mcr,'batch','script'])
matlab_cmd = ' '.join([spm_standalone,mcr,'batch'])
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

## Nipype verbosity

In [4]:
import nipype
# Optional: Use the following lines to increase verbosity of output
nipype.config.set('logging', 'workflow_level',  'INFO')
nipype.config.set('logging', 'interface_level', 'INFO')
nipype.logging.update_logging(nipype.config)

## 1 - we decode the xml file

In [5]:
flibasedir = '/scratch/user/hirsch/datadir/data_set/t0009/repos01'
resultdir = '/scratch/user/hirsch/datadir/data_results'

In [6]:
# we get the xml file in this directory
def getXmlFile(flibasedir):
    import os
    dirs = os.listdir( flibasedir )
    #print dirs

    import re
    xmlmatch = r'(.*)xml'
    for dir in dirs:
        xmlfound=re.search(xmlmatch,dir,flags=0)
        if xmlfound:
            #print xmlfound.group()
            return flibasedir + '/' + xmlfound.group()

In [7]:
xmlfile = getXmlFile(flibasedir)

print xmlfile

/scratch/user/hirsch/datadir/data_set/t0009/repos01/xml_example.xml


In [8]:
# the we decode the xmlfile
import xml.etree.ElementTree as ET
tree = ET.parse(xmlfile)
root = tree.getroot()

In [9]:
print root.tag

def getEpiInfos(xmlfile):
    result = {}
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # epibold directory
        for epibold in root.iter('EPIBOLD'):
            for child in epibold:
                print child.text
                if child.tag == 'file':
                    epidir = child.text
                    result['epidir'] = epidir
                    print epidir
                    
                # parameters
                if child.tag == 'parameters':
                    print "parameters"
                    
                    for child2 in child:
                                                     
                        # TR
                        if child2.tag == 'TR':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    tr = child3.text
                                    print tr
                                    result['TR'] = tr
                    
                        # dynamics
                        if child2.tag == 'dynamics':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    dynamics = child3.text
                                    result['dynamics'] = dynamics
                                    
                        # sliceTimingVector
                        if child2.tag == 'sliceTimingVector':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    sliceTimingVector = child3.text
                                    result['sliceTimingVector'] = sliceTimingVector
                                    
                        # nb_slices
                        if child2.tag == 'nb_slices':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    nb_slices = child3.text
                                    result['nb_slices'] = nb_slices
                
        
    except:
        print 'exception'
    return result


RS_analysis


In [10]:
epiResults = getEpiInfos(xmlfile)
print epiResults

/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2
{'sliceTimingVector': '1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30', 'dynamics': '240', 'nb_slices': '31', 'TR': '2', 'epidir': '/EPIBOLD/'}


In [11]:
def getT1file(xmlfile):
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # T1 directory and file
        for t1 in root.iter('T1'):
            for child in t1:
                #print child.text
                if child.tag == 'file':
                    t1 = child.text
                    t1 = flibasedir + t1
                    # print  t1
    except:
        print 'exception'
    return t1
    
    

In [12]:
t1 = getT1file(xmlfile)
print t1

/scratch/user/hirsch/datadir/data_set/t0009/repos01/T1/t0009_t1_s03.nii


# Start Pileline -> declaration

In [82]:
preproc = pe.Workflow(name='preproc')

## 1 - first node data grabbing by select files 

In [83]:
epidir = getEpiInfos(xmlfile)['epidir']
epidir = flibasedir + epidir

from nipype import SelectFiles, Node
templates = dict(T1=t1,
                 epi= epidir + "/" + "*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()


/EPIBOLD/
/EPIBOLD/
bold-dataset

     
parameters
2


{'T1': <undefined>, 'epi': <undefined>}

## 2 - segment T1

In [84]:
segment = pe.Node(interface=spm.NewSegment(), name='segment')

#seg.inputs.channel_files = 't0009_t1_s03.nii'
segment.inputs.channel_info =(0.001,60.0,(True,True))          
segment.inputs.sampling_distance= 3.0
# todo donner un chemin de TPM.nii comme argument de la future fct
tissue1 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 1), 1, (True,False), (False, False))
tissue2 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 2), 1, (True,False), (False, False))
tissue3 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 3), 2, (True,False), (False, False))
tissue4 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 4), 3, (True,False), (False, False))
tissue5 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 5), 4, (True,False), (False, False))
tissue6 = (('/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii', 6), 2, (False,False), (False, False))
segment.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
segment.inputs.affine_regularization = 'mni'
#segment.inputs.warping_regularization = [0.0, 0.001, 0.5, 0.05, 0.2]
segment.inputs.write_deformation_fields = [False, False]

# 2 - segment
preproc.connect(filesource,"T1" ,segment, 'channel_files')


## 2 bis - image calculator

In [85]:
def computeStructuralImage(gm_img, wm_img, mean_img):
    ''' 
    takes 3 images arguments: grey matter, white matter, and bias corrected T1 img.
    compute a binary mask with gm and wm thresholded at 0.2 combined with the mean image coming out of segment.
    used to realigne properly the EPI imgs. 
    '''

    import numpy as np     
    import nibabel as nib     
    import os 
    
    i1=nib.load(gm_img)         
    i1array=np.asarray(i1.dataobj).copy() # Avoid caching the proxy image
    
    i2=nib.load(wm_img)         
    i2array=np.asarray(i2.dataobj).copy()
    
    i3=nib.load(mean_img)         
    i3array=np.asarray(i3.dataobj).copy()
    hdr3 = i3.header
    
    gi = i1array + i2array
    # threshold image gm + wm at 0.2
    gi[gi < 0.2] = 0
    # binary mask the resulting image
    gi[gi > 0.0] = 1
    
    # apply the binary mask to the bias corrected T1
    struct_image_array = np.multiply(gi, i3array)
    
    # crate the resulting image
    struct_image = nib.Nifti1Image(struct_image_array, np.eye(4))
    
    out_file = os.path.join(os.getcwd(), 'struct_image.nii')
    nib.save(struct_image, out_file)
        
    return out_file

def regexfilter(files_list,patern):
    import re
    for f in files_list:
        for g in f:
            if re.search(patern, str(g)):
                res = g
    return res

    
    
    
    

import os
import re
res = [f for f in os.listdir(path) if re.search(r'(abc|123|a1b).*\.txt$', f)]
for f in res:
    print f

In [86]:
from nipype.interfaces.utility import Function, IdentityInterface

computeStructuralImage = Node(Function(input_names=['gm_img', 'wm_img', 'mean_img'],
                                output_names=['out_file'],
                                function=computeStructuralImage),
                                name='computeStructuralImage')

# 2 bis - image calculator -> to ger a nice structurak images to get good registration results
# input_names=['gm_img', 'wm_img', 'bias_corrected_img'],
preproc.connect(segment, ('native_class_images', regexfilter,r'.*c1.*.nii') ,computeStructuralImage, 'gm_img')
preproc.connect(segment, ('native_class_images', regexfilter,r'.*c2.*.nii') ,computeStructuralImage, 'wm_img')
# todo retrouver eventuellement la mean image je ne sais pas pourquoi elle n est pas produite par segment
preproc.connect(filesource,"T1" ,computeStructuralImage, 'mean_img')

## 3 - slice timing of epibold images

In [87]:
l = epiResults['sliceTimingVector'].split()
lint =[]
for i in l:
    lint.append(int(i))
print lint

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]


In [88]:
from nipype.interfaces.spm import SliceTiming

st = pe.Node(interface=spm.SliceTiming(), name='st')
#st.inputs.in_files = 'functional.nii'
st.inputs.num_slices = int(epiResults['nb_slices'])
st.inputs.time_repetition = float(epiResults['TR'])
st.inputs.time_acquisition = float(epiResults['TR']) - float(epiResults['TR'])/2.0
# todo check previous line
print st.inputs.time_acquisition
st.inputs.slice_order = lint
st.inputs.ref_slice = int(epiResults['nb_slices'])
#st.inputs.jobtype = 'estwrite'

preproc.connect(filesource,"epi" ,st, 'in_files')

1.0


## 4 - realign of epibold images

In [89]:
realign = pe.Node(interface=spm.Realign(), name='realign')

realign.inputs.register_to_mean = True
realign.inputs.quality = 0.9
realign.inputs.separation = 4
realign.inputs.fwhm = 5
realign.inputs.interp = 2
realign.inputs.wrap = [0, 0, 0]
# essai au defaut 2, 1 realign.inputs.write_which = [0, 1]
realign.inputs.write_which = [2, 1]
realign.inputs.write_interp = 4
realign.inputs.wrap = [0, 0, 0]
realign.inputs.write_mask = True
realign.inputs.out_prefix = 'r'
realign.inputs.jobtype = 'estwrite'

preproc.connect(st,"timecorrected_files" ,realign, 'in_files')

## 5 - Coregistration

In [90]:
# spm.Coregister()
coregister = pe.Node(interface=spm.Coregister(), name='coregister' )

coregister.inputs.cost_function = 'nmi'
coregister.inputs.separation = [4.0, 2.0]
coregister.inputs.jobtype = 'estwrite'
coregister.inputs.tolerance = [0.02, 0.02, 0.02, 0.001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001]
coregister.inputs.fwhm = [7.0, 7.0]
coregister.inputs.write_interp = 4
coregister.inputs.write_wrap = [0, 0, 0]
coregister.inputs.write_mask = False 
coregister.inputs.out_prefix = 'c'
coregister.inputs.jobtype = 'estwrite'

preproc.connect(computeStructuralImage,"out_file" ,coregister, 'source')
preproc.connect(realign,"mean_image" ,coregister, 'target')
preproc.connect(realign,"realigned_files" ,coregister, 'apply_to_files')

## 6 - Normalize

In [91]:
# passage à normalize sans 12 non on repasse en 12 et on prend des param de pierre y
norm12 = pe.Node(interface=spm.Normalize12(), name='norm12')

norm12.inputs.bias_regularization = 0.0001
norm12.inputs.bias_fwhm = 60
# todo putthis parameter in args of a fct
# norm12.inputs.tpm = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii'
norm12.inputs.affine_regularization_type = 'mni'
norm12.inputs.sampling_distance = 3.0
norm12.inputs.write_bounding_box = [[-90.0, -126.0, -72.0],[ 90.0, 90.0, 108.0]]
norm12.inputs.write_voxel_sizes = [2.0, 2.0, 2.0]
norm12.inputs.write_interp = 4 
norm12.inputs.jobtype = 'estwrite'
norm12.inputs.use_v8struct = True
#norm12.inputs.out_prefix = 'w'

preproc.connect(segment,'bias_corrected_images' ,norm12, 'image_to_align')
preproc.connect(coregister,"coregistered_files" ,norm12, 'apply_to_files')

In [92]:
# deuxieme normalisation pour normaliser les masks wm et lcf
norm12bis = pe.Node(interface=spm.Normalize12(), name='norm12bis')

norm12bis.inputs.bias_regularization = 0.0001
norm12bis.inputs.bias_fwhm = 60
# todo putthis parameter in args of a fct
# norm12.inputs.tpm = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM.nii'
norm12bis.inputs.affine_regularization_type = 'mni'
norm12bis.inputs.sampling_distance = 3.0
norm12bis.inputs.write_bounding_box = [[-90.0, -126.0, -72.0],[ 90.0, 90.0, 108.0]]
norm12bis.inputs.write_voxel_sizes = [2.0, 2.0, 2.0]
norm12bis.inputs.write_interp = 4 
norm12bis.inputs.jobtype = 'estwrite'
norm12bis.inputs.use_v8struct = True
#norm12bis.inputs.out_prefix = 'w'

# rajout de normalisation des tissus
preproc.connect(segment,'bias_corrected_images' ,norm12bis, 'image_to_align')
preproc.connect(segment,"native_class_images" ,norm12bis, 'apply_to_files')

In [93]:
preproc.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpuftLcP/preproc/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node segment in dir: /tmp/tmp5nskvw/preproc/segment
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node computeStructuralImage in dir: /tmp/tmpRubAsj/preproc/computeStructuralImage
INFO:workflow:Executing node norm12bis in dir: /tmp/tmpF4vWCG/preproc/norm12bis
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node st in dir: /tmp/tmpW_txtq/preproc/st
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node realign in dir: /tmp/tmpeL0uAp/preproc/realign
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node coregister in dir: /tmp/tmp351OOy/preproc/coregister
INFO:workflow:Runtime memory and threads stats unavailabl

## 7 -fsl merge

In [ ]:
from nipype.interfaces.fsl import Merge
fsl_merge = pe.Node(interface=Merge(), name="fsl_merge")     
 
fsl_merge.inputs.dimension = 't'
fsl_merge.inputs.output_type = 'NIFTI_GZ'
# todo recuperer le TR deja exploité du xml
fsl_merge.inputs.tr = 2.0
#fsl_merge.inputs.force_even = False     
fsl_merge.inputs.ignore_exception = False     
#fsl_merge.inputs.sort = False

## 8 - low pass high pass

In [ ]:
from nipype.interfaces import afni

bandpass = pe.Node(interface=afni.Bandpass(), name='bandpass')
# bandpass.inputs.in_file = 'functional.nii'

bandpass.inputs.highpass = 0.005
bandpass.inputs.lowpass = 0.2
bandpass.inputs.automask = False     
bandpass.inputs.environ = {}
bandpass.inputs.outputtype = 'NIFTI_GZ'     
bandpass.inputs.terminal_output = 'stream'

# connection du pipeline

In [ ]:

preproc.connect(norm12,  'normalized_files', fsl_merge, 'in_files')

preproc.connect(fsl_merge,  'merged_file', bandpass, 'in_file')


## Put the results in the datasink

In [ ]:
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'

structural results

In [ ]:
# segment results
preproc.connect(segment,  'normalized_class_images', datasink, 'structural')
preproc.connect(segment,  'bias_corrected_images' , datasink, 'structural.@par')
preproc.connect(segment,  'bias_field_images' , datasink, 'structural.@par1')
preproc.connect(segment,  'forward_deformation_field' , datasink, 'structural.@par2')
preproc.connect(segment,  'inverse_deformation_field' , datasink, 'structural.@par3')
preproc.connect(segment,  'modulated_class_images' , datasink, 'structural.mod_class')
preproc.connect(segment,  'native_class_images' , datasink, 'structural.native_class')
preproc.connect(segment,  'transformation_mat' , datasink, 'structural.@par6')
preproc.connect(segment,  'dartel_input_images' , datasink, 'structural.@par7')

# rajout de normalisation des tissues
preproc.connect(norm12bis,  'normalized_files', datasink, 'structural.norm_files')
preproc.connect(norm12bis,  'normalized_image', datasink, 'structural.norm_image')

functional results

In [ ]:
# slice timing results
preproc.connect(st,  'timecorrected_files', datasink, 'functionnal')

# realign mean_image and realignment_parameters
preproc.connect(realign,  'mean_image', datasink, 'functionnal.@par')
preproc.connect(realign,  'realignment_parameters', datasink, 'functionnal.@par1')
preproc.connect(realign,  'realigned_files', datasink, 'functionnal.@par2')

preproc.connect(coregister,  'coregistered_files', datasink, 'functionnal.@par3')

preproc.connect(norm12,  'normalized_files', datasink, 'functionnal.norm_files')
preproc.connect(norm12,  'normalized_image', datasink, 'functionnal.norm_image')



preproc.connect(fsl_merge,  'merged_file', datasink, 'functionnal.@par6')

preproc.connect(bandpass,  'out_file', datasink, 'functionnal.@par7')

# THE RUN

In [ ]:
preproc.run()

## the end

# Brouillons

In [28]:
%pwd
%ls

arimav1.ipynb
bandpassv1.ipynb
bandpassv2.ipynb
bandpassv3.ipynb
bandpassv4.ipynb
bandpassv4.py
crash-20170309-124416-hirsch-norm12-3cfb0fd7-18cb-4810-ab7b-538590a2f7f8.pklz
crash-20170314-121031-hirsch-bandpass-80dac15d-8ef0-47e0-8ece-c35f12d1fd6a.pklz
crash-20170316-102433-hirsch-norm12bis-d218eac5-ec62-4fc1-94fd-7cd694989c71.pklz
crash-20170316-183350-hirsch-datasink-2121ffc6-075d-4325-aed2-64824193f506.pklz
crash-20170316-184003-hirsch-datasink-05c34874-77ed-4b0e-801f-1c7180916024.pklz
crash-20170316-184615-hirsch-datasink-c8c536a8-d631-4ffc-8d2f-368af0d774d4.pklz
crash-20170316-190454-hirsch-datasink-4a60535d-d75b-4c47-86ab-ccca48e2b2e9.pklz
crash-20170317-105444-hirsch-computeMoco-352fa397-b097-46dd-92d8-a92264a119bb.pklz
crash-20170317-105916-hirsch-computeMoco-b725d32b-af16-4ae5-a434-0dcce8facba0.pklz
crash-20170317-114854-hirsch-computeMoco-0576aa30-4b34-4296-81fd-4a0fb0434464.pklz
crash-20170317-120411-hirsch-computeMoco-faaf29ec-0e2a-4d22-a6d5-654f69e6f2a9.pklz
crash-2017031

In [30]:
%pwd

u'/homes_unix/hirsch/_ipnotebooks'

In [32]:
path = u'/homes_unix/hirsch/_ipnotebooks'
import os
import re
res = [f for f in os.listdir(path) if re.search(r'ni.*v5.ipynb', f)]
for f in res:
    print f

nipypev5.ipynb


In [33]:
os.listdir(path)

[u'.ipynb_checkpoints',
 u'.git',
 u'nipypev1.ipynb',
 u'nipypev2.ipynb',
 u'nipypev3.ipynb',
 u'pyscript.m',
 u'preproc',
 u'nipypev3.py',
 u'nipypev4.ipynb',
 u'nipypev5.ipynb',
 u'nipypev5(2).py',
 u'try_nibabel_v2.ipynb',
 u'crash-20170321-103202-hirsch-computeStructuralImage-16ad5bed-bdee-43e8-b95e-e258179c033a.pklz',
 u'nipypev6.ipynb',
 u'crash-20170309-124416-hirsch-norm12-3cfb0fd7-18cb-4810-ab7b-538590a2f7f8.pklz',
 u'nipypev7.ipynb',
 u'crash-20170314-121031-hirsch-bandpass-80dac15d-8ef0-47e0-8ece-c35f12d1fd6a.pklz',
 u'nipypev7.py',
 u'arimav1.ipynb',
 u'nipypev7-special-togetortho-v1.ipynb',
 u'bandpassv1.ipynb',
 u'ortho.txt',
 u'nipypev8.ipynb',
 u'crash-20170316-102433-hirsch-norm12bis-d218eac5-ec62-4fc1-94fd-7cd694989c71.pklz',
 u'bandpassv2.ipynb',
 u'prebandpass',
 u'bandpassv3.ipynb',
 u'crash-20170316-183350-hirsch-datasink-2121ffc6-075d-4325-aed2-64824193f506.pklz',
 u'crash-20170316-184003-hirsch-datasink-05c34874-77ed-4b0e-801f-1c7180916024.pklz',
 u'crash-201703

In [29]:
from nipype.interfaces.fsl.maths import MathsCommand






MathsCommand().help()

Wraps command **fslmaths**


Inputs::

	[Mandatory]
	in_file: (an existing file name)
		image to operate on
		flag: %s, position: 2

	[Optional]
	args: (a string)
		Additional parameters to the command
		flag: %s
	environ: (a dictionary with keys which are a value of type 'str' and
		 with values which are a value of type 'str', nipype default value:
		 {})
		Environment variables
	ignore_exception: (a boolean, nipype default value: False)
		Print an error message instead of throwing an exception in case the
		interface fails to run
	internal_datatype: ('float' or 'char' or 'int' or 'short' or 'double'
		 or 'input')
		datatype to use for calculations (default is float)
		flag: -dt %s, position: 1
	nan2zeros: (a boolean)
		change NaNs to zeros before doing anything
		flag: -nan, position: 3
	out_file: (a file name)
		image to write
		flag: %s, position: -2
	output_datatype: ('float' or 'char' or 'int' or 'short' or 'double'
		 or 'input')
		datatype to use for output (default uses inpu

In [66]:
spm.Normalize12().help()

uses SPM12's new Normalise routine for warping an image to a template.
Spatial normalisation is now done via the segmentation routine (which was
known as ``New Segment`` in SPM8). Note that the normalisation in SPM12
is done towards a file containing multiple tissue probability maps, which
was not the cass in SPM8.

http://www.fil.ion.ucl.ac.uk/spm/doc/manual.pdf#page=49

Examples
--------
>>> import nipype.interfaces.spm as spm
>>> norm12 = spm.Normalize12()
>>> norm12.inputs.image_to_align = 'structural.nii'
>>> norm12.inputs.apply_to_files = 'functional.nii'
>>> norm12.run() # doctest: +SKIP

Inputs::

	[Mandatory]
	deformation_file: (a file name)
		file y_*.nii containing 3 deformation fields for the deformation in
		x, y and z dimension
		mutually_exclusive: image_to_align, tpm
	image_to_align: (an existing file name)
		file to estimate normalization parameters with
		mutually_exclusive: deformation_file

	[Optional]
	affine_regularization_type: ('mni' or 'size' or 'none')
		mni, 

In [ ]:
spm.Realign().help()


In [ ]:
import statsmodels.tsa.arima_model.ARIMA

In [ ]:
afni.Bandpass().help()


In [ ]:
from statsmodels import  tsa

In [ ]:
spm

.Normalize12().help()

In [ ]:
spm.Coregister.help()

In [ ]:
spm.Realign().help()

In [ ]:
spm.Realign().input_spec()

In [ ]:
l = epiResults['sliceTimingVector'].split()
lint =[]
for i in l:
    lint.append(int(i))
print lint

In [ ]:
float(epiResults['TR']) - float(epiResults['TR'])/2.0

In [ ]:
spm.SliceTiming().help()

In [ ]:
spm.SliceTiming().input_spec()

In [ ]:
bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = <undefined>
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = <undefined>
native_class_images = <undefined>
normalized_class_images = <undefined>
transformation_mat = <undefined>

In [ ]:
spm.NewSegment.help()

In [ ]:
spm.NewSegment.input_spec()

In [17]:
spm.NewSegment.output_spec()


bias_corrected_images = <undefined>
bias_field_images = <undefined>
dartel_input_images = <undefined>
forward_deformation_field = <undefined>
inverse_deformation_field = <undefined>
modulated_class_images = <undefined>
native_class_images = <undefined>
normalized_class_images = <undefined>
transformation_mat = <undefined>

In [ ]:
/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/Atlases/TPM

In [ ]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD
%pwd